In [1]:
import pandas as pd
import numpy as np
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures / Original Film,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]
...,...,...,...,...,...,...
240,DECEMBER,25,Spies in Disguise,20th Century Fox / Blue Sky Studios / Chernin ...,"Nick Bruno, Troy Quane (directors); Brad Copel...",[132]
241,DECEMBER,25,Little Women,Columbia Pictures / Regency Enterprises,Greta Gerwig (director/screenplay); Saoirse Ro...,[224]
242,DECEMBER,25,1917,Universal Pictures / DreamWorks Pictures,Sam Mendes (director/screenplay); Krysty Wilso...,[225]
243,DECEMBER,25,Just Mercy,Warner Bros. Pictures / Participant Media,"Destin Daniel Cretton (director/screenplay), A...",[226]


In [9]:
df_2019 = df[['Title']]

In [10]:
!pip install tmdbv3api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from tmdbv3api import TMDb
import json
tmdb = TMDb()
tmdb.api_key = '2d71ac9f8f76e2fe996cdc92cab9f9cd'

In [12]:
from tmdbv3api import Movie
import json
tmdb_movie = Movie() 

In [13]:
def ids(x):
  result = tmdb_movie.search(x)
  if not result:
    return np.NaN
  else:
    return result[0].id

In [14]:
df_2019['id'] = df_2019['Title'].map(lambda x: ids(str(x)))

<ipython-input-14-94292c00d2b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['id'] = df_2019['Title'].map(lambda x: ids(str(x)))


In [15]:
df_2019[df_2019['id'].isnull()]

,Title,id


In [16]:
def genres(movie_id):
    genres=[]
  
    movie_details = tmdb_movie.details(movie_id)
    movie_genres = movie_details.genres
    if movie_genres:
          genre_str = " " 
          for i in range(0,len(movie_genres)):
              genres.append(movie_genres[i]['name'])
          return genre_str.join(genres)
    else:
          return np.NaN


In [17]:
df_2019['genres'] = df_2019['id'].map(lambda x: genres(int(x)))

<ipython-input-17-58562f9efbf9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['genres'] = df_2019['id'].map(lambda x: genres(int(x)))


In [18]:
df_2019

,Title,id,genres
0,Escape Room,522681,Horror Thriller Mystery
1,Rust Creek,561362,Thriller Drama Action Crime
2,American Hangman,567738,Thriller
3,A Dog's Way Home,508763,Drama Adventure Family
4,The Upside,440472,Comedy Drama
...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family
241,Little Women,331482,Drama Romance History
242,1917,530915,War Drama History
243,Just Mercy,522212,Drama Crime History


In [19]:
def keywords(movie_id):
    keywords=[]

    movie_keywords = tmdb_movie.keywords(movie_id)
    #print(movie_keywords)
    if movie_keywords:
          #print(movie_keywords['keywords'])
          return movie_keywords['keywords']
    else:
          return np.NaN

In [20]:
df_2019['keywords'] = df_2019['id'].map(lambda x: keywords(int(x)))

<ipython-input-20-4ddaff3da3d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['keywords'] = df_2019['id'].map(lambda x: keywords(int(x)))


In [21]:
df_2019

,Title,id,genres,keywords
0,Escape Room,522681,Horror Thriller Mystery,"[[id, name], [id, name], [id, name], [id, name..."
1,Rust Creek,561362,Thriller Drama Action Crime,"[[id, name], [id, name], [id, name], [id, name..."
2,American Hangman,567738,Thriller,"[[id, name], [id, name], [id, name], [id, name..."
3,A Dog's Way Home,508763,Drama Adventure Family,"[[id, name], [id, name], [id, name], [id, name..."
4,The Upside,440472,Comedy Drama,"[[id, name], [id, name], [id, name], [id, name..."
...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,"[[id, name], [id, name], [id, name], [id, name]]"
241,Little Women,331482,Drama Romance History,"[[id, name], [id, name], [id, name], [id, name..."
242,1917,530915,War Drama History,"[[id, name], [id, name], [id, name], [id, name..."
243,Just Mercy,522212,Drama Crime History,"[[id, name], [id, name], [id, name], [id, name..."


In [22]:
df_2019['keywords'] = df_2019['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

<ipython-input-22-b4c6f109a8b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['keywords'] = df_2019['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])


In [23]:
s = df_2019.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

<ipython-input-23-1a36d7a7fabb>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_2019.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)


based on novel or book    27
based on true story       24
woman director            23
sequel                    22
biography                 19
Name: keyword, dtype: int64

In [24]:
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet

In [25]:
s = s[s > 1]
stemmer = SnowballStemmer('english')

In [26]:
def keywordsFiltered(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [27]:
df_2019['keywords'] = df_2019['keywords'].apply(keywordsFiltered)
df_2019['keywords'] = df_2019['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])

<ipython-input-27-3a21dfca478a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['keywords'] = df_2019['keywords'].apply(keywordsFiltered)
<ipython-input-27-3a21dfca478a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['keywords'] = df_2019['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])


In [28]:
df_2019

,Title,id,genres,keywords
0,Escape Room,522681,Horror Thriller Mystery,"[chicago, illinoi, clue]"
1,Rust Creek,561362,Thriller Drama Action Crime,"[wood, murder, surviv, meth lab, woman director]"
2,American Hangman,567738,Thriller,[kidnap]
3,A Dog's Way Home,508763,Drama Adventure Family,"[based on novel or book, new mexico, colorado,..."
4,The Upside,440472,Comedy Drama,"[based on true stori, remak, interracial frien..."
...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,[spi]
241,Little Women,331482,Drama Romance History,"[new york c, sibling relationship, based on no..."
242,1917,530915,War Drama History,[world war i]
243,Just Mercy,522212,Drama Crime History,"[based on true stori, lawyer, racism, 1980s]"


In [29]:
df_2019['keywords'] = df_2019['keywords'].map(lambda x: " ".join(x))
df_2019['keywords']

<ipython-input-29-5b4681efb2f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['keywords'] = df_2019['keywords'].map(lambda x: " ".join(x))


0                                  chicago, illinoi clue
1             wood murder surviv meth lab woman director
2                                                 kidnap
3      based on novel or book new mexico colorado bas...
4       based on true stori remak interracial friendship
                             ...                        
240                                                  spi
241    new york c sibling relationship based on novel...
242                                          world war i
243              based on true stori lawyer racism 1980s
244                                prison woman director
Name: keywords, Length: 245, dtype: object

In [30]:
def credits(movie_id):
    cast=[]
  
    movie_credits = tmdb_movie.credits(movie_id)
    if movie_credits['cast'] and movie_credits['crew']:
            for i in movie_credits['crew']:
                if i.get('job') == 'Director':
                    cast.append(i.get('name'))
                    break
            for i in range(0, len(movie_credits['cast'])):
                cast.append(movie_credits['cast'][i]['name'])
            return cast
    else:
            return np.NaN

In [31]:
df_2019['cast'] = df_2019['id'].map(lambda x: credits(int(x)))

<ipython-input-31-6d63cbabf824>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['cast'] = df_2019['id'].map(lambda x: credits(int(x)))


In [32]:
df_2019

,Title,id,genres,keywords,cast
0,Escape Room,522681,Horror Thriller Mystery,"chicago, illinoi clue","[Adam Robitel, Taylor Russell, Logan Miller, J..."
1,Rust Creek,561362,Thriller Drama Action Crime,wood murder surviv meth lab woman director,"[Jen McGowan, Hermione Corfield, Jay Paulson, ..."
2,American Hangman,567738,Thriller,kidnap,"[Wilson Coneybeare, Donald Sutherland, Vincent..."
3,A Dog's Way Home,508763,Drama Adventure Family,based on novel or book new mexico colorado bas...,"[Charles Martin Smith, Ashley Judd, Jonah Haue..."
4,The Upside,440472,Comedy Drama,based on true stori remak interracial friendship,"[Neil Burger, Kevin Hart, Bryan Cranston, Nico..."
...,...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,spi,"[Troy Quane, Will Smith, Tom Holland, Ben Mend..."
241,Little Women,331482,Drama Romance History,new york c sibling relationship based on novel...,"[Greta Gerwig, Saoirse Ronan, Florence Pugh, E..."
242,1917,530915,War Drama History,world war i,"[Sam Mendes, George MacKay, Dean-Charles Chapm..."
243,Just Mercy,522212,Drama Crime History,based on true stori lawyer racism 1980s,"[Destin Daniel Cretton, Michael B. Jordan, Bri..."


In [33]:
df_2019['cast'] = df_2019['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
df_2019['cast'] = df_2019['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

<ipython-input-33-2b3497e7ac16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['cast'] = df_2019['cast'].apply(lambda x: [i for i in x] if isinstance(x, list) else [])
<ipython-input-33-2b3497e7ac16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019['cast'] = df_2019['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])


In [34]:
df_2019

,Title,id,genres,keywords,cast
0,Escape Room,522681,Horror Thriller Mystery,"chicago, illinoi clue","[adamrobitel, taylorrussell, loganmiller, jaye..."
1,Rust Creek,561362,Thriller Drama Action Crime,wood murder surviv meth lab woman director,"[jenmcgowan, hermionecorfield, jaypaulson, sea..."
2,American Hangman,567738,Thriller,kidnap,"[wilsonconeybeare, donaldsutherland, vincentka..."
3,A Dog's Way Home,508763,Drama Adventure Family,based on novel or book new mexico colorado bas...,"[charlesmartinsmith, ashleyjudd, jonahhauer-ki..."
4,The Upside,440472,Comedy Drama,based on true stori remak interracial friendship,"[neilburger, kevinhart, bryancranston, nicolek..."
...,...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,spi,"[troyquane, willsmith, tomholland, benmendelso..."
241,Little Women,331482,Drama Romance History,new york c sibling relationship based on novel...,"[gretagerwig, saoirseronan, florencepugh, emma..."
242,1917,530915,War Drama History,world war i,"[sammendes, georgemackay, dean-charleschapman,..."
243,Just Mercy,522212,Drama Crime History,based on true stori lawyer racism 1980s,"[destindanielcretton, michaelb.jordan, brielar..."


In [35]:
df_2019['director'] = df_2019['cast'].map(lambda x:x[0] if len(x) >=1 else x)

In [36]:
df_2019

,Title,id,genres,keywords,cast,director
0,Escape Room,522681,Horror Thriller Mystery,"chicago, illinoi clue","[adamrobitel, taylorrussell, loganmiller, jaye...",adamrobitel
1,Rust Creek,561362,Thriller Drama Action Crime,wood murder surviv meth lab woman director,"[jenmcgowan, hermionecorfield, jaypaulson, sea...",jenmcgowan
2,American Hangman,567738,Thriller,kidnap,"[wilsonconeybeare, donaldsutherland, vincentka...",wilsonconeybeare
3,A Dog's Way Home,508763,Drama Adventure Family,based on novel or book new mexico colorado bas...,"[charlesmartinsmith, ashleyjudd, jonahhauer-ki...",charlesmartinsmith
4,The Upside,440472,Comedy Drama,based on true stori remak interracial friendship,"[neilburger, kevinhart, bryancranston, nicolek...",neilburger
...,...,...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,spi,"[troyquane, willsmith, tomholland, benmendelso...",troyquane
241,Little Women,331482,Drama Romance History,new york c sibling relationship based on novel...,"[gretagerwig, saoirseronan, florencepugh, emma...",gretagerwig
242,1917,530915,War Drama History,world war i,"[sammendes, georgemackay, dean-charleschapman,...",sammendes
243,Just Mercy,522212,Drama Crime History,based on true stori lawyer racism 1980s,"[destindanielcretton, michaelb.jordan, brielar...",destindanielcretton


In [37]:
df_2019['actor1']=df_2019['cast'].map(lambda x:x[1] if len(x) >=2 else x)
df_2019['actor2']=df_2019['cast'].map(lambda x:x[2] if len(x) >=3 else x)
df_2019['actor3']=df_2019['cast'].map(lambda x:x[3] if len(x) >=4 else x)

In [38]:
df_2019

,Title,id,genres,keywords,cast,director,actor1,actor2,actor3
0,Escape Room,522681,Horror Thriller Mystery,"chicago, illinoi clue","[adamrobitel, taylorrussell, loganmiller, jaye...",adamrobitel,taylorrussell,loganmiller,jayellis
1,Rust Creek,561362,Thriller Drama Action Crime,wood murder surviv meth lab woman director,"[jenmcgowan, hermionecorfield, jaypaulson, sea...",jenmcgowan,hermionecorfield,jaypaulson,seano'bryan
2,American Hangman,567738,Thriller,kidnap,"[wilsonconeybeare, donaldsutherland, vincentka...",wilsonconeybeare,donaldsutherland,vincentkartheiser,oliverdennis
3,A Dog's Way Home,508763,Drama Adventure Family,based on novel or book new mexico colorado bas...,"[charlesmartinsmith, ashleyjudd, jonahhauer-ki...",charlesmartinsmith,ashleyjudd,jonahhauer-king,edwardjamesolmos
4,The Upside,440472,Comedy Drama,based on true stori remak interracial friendship,"[neilburger, kevinhart, bryancranston, nicolek...",neilburger,kevinhart,bryancranston,nicolekidman
...,...,...,...,...,...,...,...,...,...
240,Spies in Disguise,431693,Animation Action Adventure Comedy Family,spi,"[troyquane, willsmith, tomholland, benmendelso...",troyquane,willsmith,tomholland,benmendelsohn
241,Little Women,331482,Drama Romance History,new york c sibling relationship based on novel...,"[gretagerwig, saoirseronan, florencepugh, emma...",gretagerwig,saoirseronan,florencepugh,emmawatson
242,1917,530915,War Drama History,world war i,"[sammendes, georgemackay, dean-charleschapman,...",sammendes,georgemackay,dean-charleschapman,markstrong
243,Just Mercy,522212,Drama Crime History,based on true stori lawyer racism 1980s,"[destindanielcretton, michaelb.jordan, brielar...",destindanielcretton,michaelb.jordan,brielarson,jamiefoxx


In [39]:
df_2019= df_2019.loc[:,['id','director','actor1','actor2','actor3','genres','keywords','Title']]

In [40]:
df_2019 = df_2019.rename(columns={'Title':'title'})

In [41]:
df_2019['title'] = df_2019['title'].str.lower()
df_2019['actor1'] = df_2019['actor1'].str.lower()
df_2019['actor2'] = df_2019['actor2'].str.lower()
df_2019['actor3'] = df_2019['actor3'].str.lower()
# data['director'] = data['director'].str.lower()
df_2019['comb'] = df_2019['genres'] + ' ' + df_2019['keywords'] + ' '  + df_2019['actor1'] + ' '  + df_2019['actor2'] + ' '  + df_2019['actor3'] + ' ' + df_2019['director']

In [42]:
df_2019

,id,director,actor1,actor2,actor3,genres,keywords,title,comb
0,522681,adamrobitel,taylorrussell,loganmiller,jayellis,Horror Thriller Mystery,"chicago, illinoi clue",escape room,"Horror Thriller Mystery chicago, illinoi clue ..."
1,561362,jenmcgowan,hermionecorfield,jaypaulson,seano'bryan,Thriller Drama Action Crime,wood murder surviv meth lab woman director,rust creek,Thriller Drama Action Crime wood murder surviv...
2,567738,wilsonconeybeare,donaldsutherland,vincentkartheiser,oliverdennis,Thriller,kidnap,american hangman,Thriller kidnap donaldsutherland vincentkarthe...
3,508763,charlesmartinsmith,ashleyjudd,jonahhauer-king,edwardjamesolmos,Drama Adventure Family,based on novel or book new mexico colorado bas...,a dog's way home,Drama Adventure Family based on novel or book ...
4,440472,neilburger,kevinhart,bryancranston,nicolekidman,Comedy Drama,based on true stori remak interracial friendship,the upside,Comedy Drama based on true stori remak interra...
...,...,...,...,...,...,...,...,...,...
240,431693,troyquane,willsmith,tomholland,benmendelsohn,Animation Action Adventure Comedy Family,spi,spies in disguise,Animation Action Adventure Comedy Family spi w...
241,331482,gretagerwig,saoirseronan,florencepugh,emmawatson,Drama Romance History,new york c sibling relationship based on novel...,little women,Drama Romance History new york c sibling relat...
242,530915,sammendes,georgemackay,dean-charleschapman,markstrong,War Drama History,world war i,1917,War Drama History world war i georgemackay dea...
243,522212,destindanielcretton,michaelb.jordan,brielarson,jamiefoxx,Drama Crime History,based on true stori lawyer racism 1980s,just mercy,Drama Crime History based on true stori lawyer...


In [43]:
df_2019.to_csv('movies2019.csv',index=False)

In [44]:
from google.colab import files
files.download("movies2019.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>